# Práctica Guiada - Técnicas para datasets desbalanceados

El problema de los datasets desbalanceados se encuentra en una amplia variedad de problemas de Machine Learning como la detección de comentarios hechos a través de bots, el diagnóstico de enfermedades o la búsqueda de transacciones fraudulentas en un sistema.

Existen dos grandes enfoques para abordar datasets que se encuentran desbalanceados:

1.  Hacer un resampling de la muestra, para entrenar al algoritmo con proporciones similares
2.  Incorporar el desbalance a la función de costos del algoritmo para que tenga incentivos a elegir los parámetros que mejor discriminan la clase minoritaria.

## Clasificación sobre datos desbalanceados

A continuación presentamos un dataset de la empresa americana Lending Club, que se dedica a proveer servicios financieros para distintos segmentos. 

A continuación vamos a utilizar información abierta del portal de la empresa para intentar predecir cuáles de los créditos terminan en default. Para más información sobre el datset pueden ingresar <a href='https://www.lendingclub.com/info/download-data.action'> aquí </a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
loans = pd.read_csv('../Data/loans.csv',low_memory=False)

In [3]:
loans.head()

,grade,home_ownership,emp_length_num,sub_grade,short_emp,dti,term,purpose,int_rate,last_delinq_none,last_major_derog_none,revol_util,total_rec_late_fee,payment_inc_ratio,bad_loans
0,B,RENT,11,B2,0,27.65,36 months,credit_card,10.65,1,1,83.7,0.00,8.14350,0
1,C,RENT,1,C4,1,1.00,60 months,car,15.27,1,1,9.4,0.00,2.39320,1
2,C,RENT,11,C5,0,8.72,36 months,small_business,15.96,1,1,98.5,0.00,8.25955,0
3,C,RENT,11,C1,0,20.00,36 months,other,13.49,0,1,21.0,16.97,8.27585,0
4,A,RENT,4,A4,0,11.20,36 months,wedding,7.90,1,1,28.3,0.00,5.21533,0


## Análisis exploratorio

#### Balanceo de la clase

La clase que vamos a intentar predecir es "bad loans" que indica si el préstamo fue pagado a tiempo o no. Observamos que la clase está desbalanceada, la mayoría de los préstamos se pagan a tiempo.

In [4]:
loans['bad_loans'].value_counts(normalize=True)

0    0.811185
1    0.188815
Name: bad_loans, dtype: float64

#### Valores faltantes

Inspeccionamos los valores faltantes

In [5]:
loans.isnull().sum()

grade                    0
home_ownership           0
emp_length_num           0
sub_grade                0
short_emp                0
dti                      0
term                     0
purpose                  0
int_rate                 0
last_delinq_none         0
last_major_derog_none    0
revol_util               0
total_rec_late_fee       0
payment_inc_ratio        4
bad_loans                0
dtype: int64

Notamos que la variable payment_inc_ratio tiene sólo 4 valores nulos y muestra una correlación importante con la clase.
Además sabemos que la relación cuota/ingreso es importante a al hora de evaluar la capacidad de repago. 
Eliminamos, entonces, los valores con payment_inc_ratio desconocido.

In [6]:
loans = loans[loans.payment_inc_ratio.notnull()]

In [7]:
loans.sample(10)

,grade,home_ownership,emp_length_num,sub_grade,short_emp,dti,term,purpose,int_rate,last_delinq_none,last_major_derog_none,revol_util,total_rec_late_fee,payment_inc_ratio,bad_loans
36702,E,RENT,2,E5,0,2.05,36 months,home_improvement,17.74,0,1,47.1,0.0,3.72667,0
47450,C,MORTGAGE,3,C5,0,23.04,60 months,home_improvement,17.10,0,0,52.1,0.0,13.39690,0
46734,C,MORTGAGE,6,C5,0,25.34,60 months,debt_consolidation,17.10,1,1,94.8,0.0,6.70605,1
30220,B,RENT,2,B5,0,22.74,36 months,wedding,12.53,1,1,67.1,0.0,11.45030,0
344,A,RENT,1,A4,1,1.68,60 months,debt_consolidation,7.90,1,1,10.0,0.0,4.84164,1
83720,B,MORTGAGE,11,B2,0,28.29,36 months,debt_consolidation,11.14,0,1,46.7,0.0,11.47720,0
43438,E,MORTGAGE,7,E1,0,18.60,36 months,other,21.00,0,1,67.2,0.0,3.76760,1
117781,B,RENT,3,B5,0,19.82,36 months,credit_card,12.99,0,0,72.0,0.0,4.97569,0
30881,C,MORTGAGE,2,C5,0,13.68,36 months,credit_card,14.26,1,1,80.8,0.0,11.76210,0
1310,B,RENT,2,B3,0,19.82,36 months,debt_consolidation,11.71,1,1,88.0,0.0,5.67017,0


## Modelo Predictivo: regresión logística

Vamos a utilizar Regresión Logística para predecir la clase bad_loans. 

In [8]:
# Generamos las variables dummies para los datos categóricos.
loans_enconded = pd.get_dummies(loans)

In [9]:
training_features, test_features, training_target, test_target \
                            = train_test_split(loans_enconded.drop(['bad_loans'], axis=1),\
                                    loans_enconded['bad_loans'],\
                                    test_size = .1, stratify = loans_enconded['bad_loans'],\
                                    random_state=12)

# This stratify parameter makes a split so that the proportion of values in the sample produced will be the same as 
# the proportion of values provided to parameter stratify.
# For example, if variable y is a binary categorical variable with values 0 and 1 and there are 25% of zeros and 75% of ones, 
# stratify=y will make sure that your random split has 25% of 0's and 75% of 1's.

In [10]:
# Vemos la cantidad de observaciones en el set de entrenamiento para cada clase:

np.unique(training_target, return_counts=True)

(array([0, 1], dtype=int64), array([89507, 20835], dtype=int64))

In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

training_features = scaler.fit_transform(training_features)



In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix

In [13]:
model = LogisticRegression(C=1e10, solver='lbfgs')
model.fit(training_features,training_target)

LogisticRegression(C=10000000000.0, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [14]:
# Normalizamos las features de testeo:
test_features = scaler.transform(test_features)

# Calculamos las predicciones del modelo en el set de testeo:
y_pred = model.predict(test_features)


In [15]:
# Calculamos el área debajo de la curva ROC:

roc_auc_score(test_target,y_pred)

0.5099668881506572

In [16]:
# Observamos el reporte de clasificación:

print(classification_report(test_target,y_pred))

              precision    recall  f1-score   support

           0       0.81      0.99      0.89      9946
           1       0.45      0.03      0.05      2315

    accuracy                           0.81     12261
   macro avg       0.63      0.51      0.47     12261
weighted avg       0.74      0.81      0.74     12261



In [17]:
# Observamos la matriz de confusión:

print(confusion_matrix(test_target,y_pred))

[[9865   81]
 [2250   65]]


Cuando evaluamos el modelo sobre datos no observados, el área debajo de la curva es muy cercana a 0.5. Especificamente, el recall a la hora de encontrar los "malos créditos" es  muy malo. Uno de los problemas que tenemos para hacer un buen modelo es el desbalanceo de los datos. 

Vamos a intentar dos posibles soluciones.

## Oversampling

Para aumentar la representación de la clase minoritaria vamos a hacer un oversampling utilizando el algoritmo SMOTE (Synthetic Minority Oversample) del paquete imblearn.

Este algoritmo genera nuevos datos utilizando la técnica de los K vecinos más cercanos.
Para generar un nuevo punto:

1. Se elige un punto al azar de la clase minoritaria y sus K vecinos más cercanos.
2. Se elige al azar uno de esos vecinos.
3. Se calcula el vector entre el punto seleccionado y el vecino seleccionado al azar y se lo multiplica por un número aleatorio entre 0 y 1.
4. El punto aleatorio dentro del vector es el nuevo dato para el oversampling.


In [18]:
from imblearn.over_sampling import SMOTE

# Instanciamos la clase SMOTE y realizamos el oversampling:

sm = SMOTE(random_state=12)

x_train_res, y_train_res = sm.fit_sample(training_features, training_target)

x_train_res = scaler.fit_transform(x_train_res)


In [19]:
# Vemos la cantidad de observaciones en el set de entrenamiento resampleado para cada clase:

np.unique(y_train_res, return_counts=True)

(array([0, 1], dtype=int64), array([89507, 89507], dtype=int64))

In [20]:
# Entrenamos la regresión logística con los nuevos datos rebalanceados:

model.fit(x_train_res,y_train_res)

LogisticRegression(C=10000000000.0, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [21]:
# Normalizamos los datos del set de testeo y hacemos las predicciones:

test_features = scaler.transform(test_features)
y_pred = model.predict(test_features)

In [22]:
# Calculamos el área debajo de la curva ROC:

roc_auc_score(test_target,y_pred)

0.6294048336177344

In [23]:
# Observamos el reporte de clasificación:

print(classification_report(test_target,y_pred))

              precision    recall  f1-score   support

           0       0.88      0.63      0.73      9946
           1       0.28      0.63      0.39      2315

    accuracy                           0.63     12261
   macro avg       0.58      0.63      0.56     12261
weighted avg       0.77      0.63      0.67     12261



In [24]:
# Observamos la matriz de confusión:

print(confusion_matrix(test_target,y_pred))

[[6226 3720]
 [ 850 1465]]


# Class Weights

La otra técnica que podemos utilizar para corregir el desbalance de los datos es incorporar en la función de costos del algoritmo un mayor peso para los errores de entrenamiento cometidos sobre los puntos de la clase minoritaria.

Implementamos la Regresión Logística con un parámetro que cambie la ponderación de las clases. <br />

¿Tiene sentido aplicar esta corrección junto con el algoritmo SMOTE?
<strong />

In [25]:
model = LogisticRegression(C=1e10, class_weight = 'balanced', solver='lbfgs')

# class_weight : dict or ‘balanced’, optional (default=None)
# Weights associated with classes in the form {class_label: weight}. 
# If not given, all classes are supposed to have weight one.
# The “balanced” mode uses the values of y to automatically adjust weights 
# inversely proportional to class frequencies in the 
# input data as n_samples / (n_classes * np.bincount(y)).
# Note that these weights will be multiplied with sample_weight (passed through the fit method) 
# if sample_weight is specified.

In [26]:
model.fit(training_features,training_target)

LogisticRegression(C=10000000000.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [27]:
y_pred = model.predict(test_features)

In [28]:
# Calculamos el área debajo de la curva ROC:

roc_auc_score(test_target,y_pred)

0.6292872439901168

In [29]:
# Observamos el reporte de clasificación:

print(classification_report(test_target,y_pred))

              precision    recall  f1-score   support

           0       0.87      0.70      0.78      9946
           1       0.30      0.56      0.39      2315

    accuracy                           0.68     12261
   macro avg       0.59      0.63      0.59     12261
weighted avg       0.76      0.68      0.71     12261



In [30]:
# Observamos la matriz de confusión:

print(confusion_matrix(test_target,y_pred))

[[6997 2949]
 [1030 1285]]
